In [1]:
!nvidia-smi

Mon Jun 10 20:29:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.36                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  | 00000000:01:00.0  On |                  N/A |
| 30%   46C    P5              56W / 200W |   2956MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import json
import logging

import pandas as pd
import numpy as np
import torch

from collections import Counter
from functools import partial

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix

from train_utils import data_processor, most_common_words, fix_train_common, tokenize_and_align_labels, compute_metrics

In [3]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, load_metric

from transformers import pipeline
from transformers import AutoTokenizer, RobertaTokenizerFast, DebertaV2TokenizerFast, DebertaTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, RobertaForTokenClassification, DebertaV2ForTokenClassification, DebertaForTokenClassification
from transformers import AutoModel, TrainingArguments, Trainer

from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [4]:
with open("config.json") as json_file:
    config = json.load(json_file)

In [5]:
TEST_SIZE = config['test_size']
MODEL_CHECKPOINT = config['model_checkpoint']
BATCH_SIZE = config['batch_size']
SEED = config['seed']
NUM_LAYERS = config['num_layers']

In [6]:
train = pd.read_csv('train_data_new.csv')
test = pd.read_csv('gt_test.csv')

In [7]:
train['target_labels_positions'] = train['target_labels_positions'].apply(lambda x: eval(x))

In [9]:
train['is_discount'] = [int('B-discount' in d.keys()) for d in train['target_labels_positions'].values]
train['is_value'] = [int('B-value' in d.keys()) for d in train['target_labels_positions'].values]
train['is_discount_info'] = [int('I-value' in d.keys()) for d in train['target_labels_positions'].values]

# we use it for stratification
train['label_type'] = train[['is_discount', 'is_value', 'is_discount_info']].sum(axis=1)

In [10]:
# heuristic to filter texts with empty tags
# we will use it later
train[~train['processed_text'].str.contains('скид')]['is_discount'].mean()

0.0

In [11]:
MOST_COMMON = most_common_words(train, 'processed_text', 'target_labels_positions', 5)

# filter only most common words as labels
# same logic, less noise -> better model
train = fix_train_common(train, MOST_COMMON, 'processed_text', 'target_labels_positions')

In [12]:
processed_train = data_processor(train[train['processed_text'].str.contains('скид')], 
                                 'processed_text', 'target_labels_positions')

In [13]:
ner_train, ner_test = train_test_split(processed_train, test_size=TEST_SIZE, 
                                       stratify=train[train['processed_text'].str.contains('скид')]['label_type'], 
                                       random_state=SEED)

In [14]:
proc_df = pd.DataFrame(ner_train)

In [15]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1451
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 257
    })
})

In [16]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O', 'B-discount', 'B-value', 'I-value']

In [17]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, model_max_length=512)

In [18]:
tokenized_datasets = ner_data.map(partial(tokenize_and_align_labels, tokenizer, label_list), batched=True)

Map:   0%|          | 0/1451 [00:00<?, ? examples/s]

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

In [19]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [21]:
metric = load_metric("seqeval")

/tmp/ipykernel_1686/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/home/gofat/miniconda3/lib/python3.8/site-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [22]:
# разморозка
for param in list(model.parameters())[-NUM_LAYERS:]:
    param.requires_grad = True

In [23]:
args = TrainingArguments(
    output_dir="samolet_finetuned",
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model='f1',
    load_best_model_at_end=True,
    learning_rate=1e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to='none',
)
# lr_scheduler_type='cosine',

/home/gofat/miniconda3/lib/python3.8/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, label_list, metric)
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.010211,0.414925,0.448387,0.431008,0.996377
2,0.023800,0.007284,0.516035,0.570968,0.542113,0.996473
3,0.009800,0.007376,0.544601,0.374194,0.443595,0.996855
4,0.009800,0.006916,0.598361,0.470968,0.527076,0.997020
5,0.009200,0.007888,0.566265,0.606452,0.585670,0.996603
6,0.006700,0.008646,0.689024,0.364516,0.476793,0.997124
7,0.005300,0.007875,0.614754,0.483871,0.541516,0.997133
8,0.005300,0.009074,0.537459,0.532258,0.534846,0.996620
9,0.004100,0.009532,0.575540,0.516129,0.544218,0.996846
10,0.003600,0.010233,0.560554,0.522581,0.540902,0.996725


TrainOutput(global_step=3630, training_loss=0.00868902132530843, metrics={'train_runtime': 918.4812, 'train_samples_per_second': 15.798, 'train_steps_per_second': 3.952, 'total_flos': 3779844331726272.0, 'train_loss': 0.00868902132530843, 'epoch': 10.0})

In [26]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'discount': {'precision': 0.6336206896551724,
  'recall': 0.6150627615062761,
  'f1': 0.624203821656051,
  'number': 239},
 'value': {'precision': 0.41,
  'recall': 0.5774647887323944,
  'f1': 0.47953216374269003,
  'number': 71},
 'overall_precision': 0.5662650602409639,
 'overall_recall': 0.6064516129032258,
 'overall_f1': 0.5856697819314642,
 'overall_accuracy': 0.9966028949243254}

In [27]:
trainer.save_model('samolet_finetuned_{}'.format(str(round(results['overall_f1'], 2)).replace('.', '_')))

In [28]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-discount,B-value,I-value
O,114406,85,43,126
B-discount,92,147,0,0
B-value,19,0,43,0
I-value,26,0,0,111


In [29]:
# text = ' '.join(ner_train[8]['tokens'])
text = ' '.join(ner_test[11]['tokens'])
text

'вот ээ знание наплетает на квадратные метры а она говорит что у вас короче ипотека NAME на течение часа подъеду да да да ватсап отчет с оператором хорошо спасибо два да нет для тебя третья здравствуйте хотел заехать к вам в офис на NAME алло NAME не ADDRESS NAME офис центральный у вас у остаться мне ближе а где а где девят в девят девятки найд забыл название NAME да я хотел обсудить ипотеку условия прямо сейчас в течение часа да аллодобрый день самолет меня зовут NAME как могу обращаться к вам какой у вас вопрос подскажите как обращаться к вам ваш контактный номер актуальна окончительно семьдесят два я могу вас записать через час за подъезд подскажите номер телефона семьдесят два ноль один актуально номер телефона могу информацию скажите куда лучше информацию набрать ватсап или обычное смс уведомления ватсап отлично скажите поедете наличные автомобиль на общественном транс хорошо так же хотела вам сказать что у вас будет гарантирована скидка в один процент за общение офиса именно сего

In [30]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [31]:
print(text)
print(pipe(text))

вот ээ знание наплетает на квадратные метры а она говорит что у вас короче ипотека NAME на течение часа подъеду да да да ватсап отчет с оператором хорошо спасибо два да нет для тебя третья здравствуйте хотел заехать к вам в офис на NAME алло NAME не ADDRESS NAME офис центральный у вас у остаться мне ближе а где а где девят в девят девятки найд забыл название NAME да я хотел обсудить ипотеку условия прямо сейчас в течение часа да аллодобрый день самолет меня зовут NAME как могу обращаться к вам какой у вас вопрос подскажите как обращаться к вам ваш контактный номер актуальна окончительно семьдесят два я могу вас записать через час за подъезд подскажите номер телефона семьдесят два ноль один актуально номер телефона могу информацию скажите куда лучше информацию набрать ватсап или обычное смс уведомления ватсап отлично скажите поедете наличные автомобиль на общественном транс хорошо так же хотела вам сказать что у вас будет гарантирована скидка в один процент за общение офиса именно сегод